In [88]:
import pandas as pd
import kagglehub
import numpy as np

In [ ]:

# Download latest version
# # This downloads the dataset locally and returns the folder path

path = kagglehub.dataset_download("ahmedmohamed2003/cafe-sales-dirty-data-for-cleaning-training")

print("Path to dataset files:", path)

Path to dataset files: C:\Users\qinha\.cache\kagglehub\datasets\ahmedmohamed2003\cafe-sales-dirty-data-for-cleaning-training\versions\1


In [90]:
import os 

# Read the CSV file from the downloaded directory

df = pd.read_csv(os.path.join(path , 'dirty_cafe_sales.csv'))

# Preview first 10 rows

df.head(10)

,Transaction ID,Item,Quantity,Price Per Unit,Total Spent,Payment Method,Location,Transaction Date
0,TXN_1961373,Coffee,2,2.0,4.0,Credit Card,Takeaway,2023-09-08
1,TXN_4977031,Cake,4,3.0,12.0,Cash,In-store,2023-05-16
2,TXN_4271903,Cookie,4,1.0,ERROR,Credit Card,In-store,2023-07-19
3,TXN_7034554,Salad,2,5.0,10.0,UNKNOWN,UNKNOWN,2023-04-27
4,TXN_3160411,Coffee,2,2.0,4.0,Digital Wallet,In-store,2023-06-11
5,TXN_2602893,Smoothie,5,4.0,20.0,Credit Card,NaN,2023-03-31
6,TXN_4433211,UNKNOWN,3,3.0,9.0,ERROR,Takeaway,2023-10-06
7,TXN_6699534,Sandwich,4,4.0,16.0,Cash,UNKNOWN,2023-10-28
8,TXN_4717867,NaN,5,3.0,15.0,NaN,Takeaway,2023-07-28
9,TXN_2064365,Sandwich,5,4.0,20.0,NaN,In-store,2023-12-31


# Initial data inspection

In [91]:
# Check data types and non-null counts

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Transaction ID    10000 non-null  object
 1   Item              9667 non-null   object
 2   Quantity          9862 non-null   object
 3   Price Per Unit    9821 non-null   object
 4   Total Spent       9827 non-null   object
 5   Payment Method    7421 non-null   object
 6   Location          6735 non-null   object
 7   Transaction Date  9841 non-null   object
dtypes: object(8)
memory usage: 625.1+ KB


In [92]:
# View column names

df.columns

Index(['Transaction ID', 'Item', 'Quantity', 'Price Per Unit', 'Total Spent',
       'Payment Method', 'Location', 'Transaction Date'],
      dtype='object')

In [93]:
# Summary statistics for numeric columns

df.describe()

,Transaction ID,Item,Quantity,Price Per Unit,Total Spent,Payment Method,Location,Transaction Date
count,10000,9667,9862,9821,9827,7421,6735,9841
unique,10000,10,7,8,19,5,4,367
top,TXN_1961373,Juice,5,3.0,6.0,Digital Wallet,Takeaway,UNKNOWN
freq,1,1171,2013,2429,979,2291,3022,159


In [94]:
# Count missing values per column

df.isnull().sum()

Transaction ID         0
Item                 333
Quantity             138
Price Per Unit       179
Total Spent          173
Payment Method      2579
Location            3265
Transaction Date     159
dtype: int64

In [95]:
# Check number of unique transactions

df['Transaction ID'].nunique()

10000

In [96]:
# Handle invalid placeholder values
# --------------------------------------------
# Replace 'UNKNOWN' and 'ERROR' with NaN
# Drop rows if missing values are below 5%

cols = ['Transaction ID', 'Item', 'Quantity', 'Price Per Unit', 'Total Spent',
       'Payment Method', 'Location', 'Transaction Date']

for col in cols:
    df[col] = df[col].replace(['UNKNOWN', 'ERROR'], np.nan)
    missing_ratio = df[col].isna().mean()

    if missing_ratio  < 0.05:
        df = df.dropna(subset=[col])
    else:
        print(f"The missing raio of the {col} is higher than 5% ({missing_ratio:.2%})")


The missing raio of the Item is higher than 5% (9.69%)
The missing raio of the Price Per Unit is higher than 5% (5.41%)
The missing raio of the Total Spent is higher than 5% (5.06%)
The missing raio of the Payment Method is higher than 5% (31.67%)
The missing raio of the Location is higher than 5% (39.63%)


# Data type conversions

In [97]:
# Convert quantity to integer

df['Quantity'] = df['Quantity'].astype('int16')

In [98]:
# Convert price and total spent to float

df['Price Per Unit'] = df['Price Per Unit'].astype(float)
df['Total Spent'] = df['Total Spent'].astype(float)

In [99]:
# Convert payment method to categorical type

df['Payment Method'] = df['Payment Method'].astype('category')

In [100]:
# Convert transaction date to datetime

df['Transaction Date'] = pd.to_datetime(df['Transaction Date'])

In [101]:
# Remove invalid quantities (zero or negative)

df= df[df['Quantity'] > 0]

In [102]:
# Remove invalid prices (zero or negative)

df = df[df['Price Per Unit'] > 0]


In [103]:
# Validate total spent consistency
# Calculate expected total spent

df['calculated_total'] = df['Quantity'] * df['Price Per Unit']

# Compute absolute difference

df['total_diff'] = abs(df['Total Spent'] - df['calculated_total'])

# Keep rows within acceptable rounding tolerance

df = df[df['total_diff'] < 0.01]  # allow small rounding errors

# Remove helper columns

df.drop(columns=['calculated_total', 'total_diff'], inplace=True)


In [104]:
# Count duplicate rows

df.duplicated().sum()


np.int64(0)

# Standardize categorical text values

In [105]:
# Clean item names

df['Item'] = df['Item'].str.strip().str.title()


In [106]:
# Clean payment method values

df['Payment Method'] = (
    df['Payment Method']
    .str.strip()
    .str.title()
)

# Review cleaned categories

df['Payment Method'].value_counts()


Payment Method
Digital Wallet    1888
Credit Card       1863
Cash              1847
Name: count, dtype: int64

# Outlier detection and handling


In [107]:
# Inspect quantity distribution

df['Quantity'].describe()


count    8159.000000
mean        3.018507
std         1.417945
min         1.000000
25%         2.000000
50%         3.000000
75%         4.000000
max         5.000000
Name: Quantity, dtype: float64

In [108]:
# Remove extreme quantity outliers (top 1%)

q99 = df['Quantity'].quantile(0.99)
df = df[df['Quantity'] <= q99]


# Date sanity checks

In [109]:
# Inspect date range

df['Transaction Date'].min(), df['Transaction Date'].max()


(Timestamp('2023-01-01 00:00:00'), Timestamp('2023-12-31 00:00:00'))

In [110]:
# Remove transactions with future dates

df = df[df['Transaction Date'] <= pd.Timestamp.today()]


In [111]:
# Extract day of week from transaction date

df['DayOfWeek'] = df['Transaction Date'].dt.day_name()


In [112]:
# Final dataset preview

df.head()

,Transaction ID,Item,Quantity,Price Per Unit,Total Spent,Payment Method,Location,Transaction Date,DayOfWeek
0,TXN_1961373,Coffee,2,2.0,4.0,Credit Card,Takeaway,2023-09-08,Friday
1,TXN_4977031,Cake,4,3.0,12.0,Cash,In-store,2023-05-16,Tuesday
3,TXN_7034554,Salad,2,5.0,10.0,NaN,NaN,2023-04-27,Thursday
4,TXN_3160411,Coffee,2,2.0,4.0,Digital Wallet,In-store,2023-06-11,Sunday
5,TXN_2602893,Smoothie,5,4.0,20.0,Credit Card,NaN,2023-03-31,Friday


In [113]:
# Export cleaned data for downstream analysis

df.to_csv('clean_cafe_sales.csv', index=False)
